In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import rawpy
import cv2
import math

## Resizing images to 500px the long edge by interpolation

In [ ]:
def long_edge_resize500(img):
    height = img.shape[0]
    width = img.shape[1]
    if height>width:
        new_h = 500
        new_w = int(width/height*500) 
    else:
        new_w=500
        new_h=int(height/width*500)
    print("new shape is {},{}".format(new_w, new_h))
    return cv2.resize(img, (new_w, new_h), cv2.INTER_AREA)
        
    

## Conversion from RGB to LAB and only isolate L

In [ ]:
raw_RGB = rawpy.imread("data/a0001-jmac_DSC1459.dng").postprocess()
expertA_RGB = cv2.cvtColor(cv2.imread("/media/samuel/HDD1TB/data.csail.mit.edu/graphics/fivek/img/tiff16_a/a0001-jmac_DSC1459.tif", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
#expertB_RGB = cv2.cvtColor(cv2.imread("/media/samuel/HDD1TB/data.csail.mit.edu/graphics/fivek/img/tiff16_b/a0001-jmac_DSC1459.tif", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
#expertC_RGB = cv2.cvtColor(cv2.imread("/media/samuel/HDD1TB/data.csail.mit.edu/graphics/fivek/img/tiff16_c/a0001-jmac_DSC1459.tif", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
#expertD_RGB = cv2.cvtColor(cv2.imread("/media/samuel/HDD1TB/data.csail.mit.edu/graphics/fivek/img/tiff16_d/a0001-jmac_DSC1459.tif", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
#expertE_RGB = cv2.cvtColor(cv2.imread("/media/samuel/HDD1TB/data.csail.mit.edu/graphics/fivek/img/tiff16_e/a0001-jmac_DSC1459.tif", cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)

In [ ]:
def basic_showImg(img, size=4):
    plt.figure(figsize=(size,size))
    plt.imshow(img)
    plt.show()

In [ ]:
basic_showImg(raw_RGB)
basic_showImg(expertA_RGB)

In [ ]:
raw_RGB = long_edge_resize500(raw_RGB)
expertA_RGB = long_edge_resize500(expertA_RGB)


In [ ]:
# https://docs.opencv.org/3.4/de/d25/imgproc_color_conversions.html#color_convert_rgb_lab
raw_LAB = cv2.cvtColor(raw_RGB, cv2.COLOR_RGB2Lab)
expertA_LAB = cv2.cvtColor(expertA_RGB, cv2.COLOR_RGB2Lab)

In [ ]:
raw_lum = raw_LAB[:,:,0]
expertA_lum = expertA_LAB[:,:,0]

In [ ]:
raw_lum

In [ ]:
expertA_lum

In [ ]:
raw_lum/255*100